In [1]:
!pip install gensim

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
import nltk 
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from gensim import matutils
from gensim import models
import scipy.sparse
from nltk import word_tokenize, pos_tag
from wordcloud import WordCloud
from sklearn.feature_extraction import text 

In [3]:
columns_name = ["target", "ids", "date", "flag", "user", "text"]  # Column names to be used
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = 'latin-1', names = columns_name) # Readinf the dataset


# Due to the size of the the file, we select the first 50000 tweets and then reshuffle them later
df_main = df[0:100]
df_main = shuffle(df_main)

In [4]:
df_main = df_main.drop(['ids', 'date', 'flag', 'user'], axis=1)

In [5]:
corpus = []
for i in range(0,len(df_main)):
    review=re.sub('@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+',' ',df_main['text'][i]) #delete special symbol
    review=review.lower() #changing all characters to lowercase
    review=review.split()
    review=' '.join(review) #joins  into one string
    corpus.append(review)
    
df_main.text = corpus

In [6]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df_main.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df_main.index
data_dtm

,10,12,1st,50,5am,92,able,account,actually,added,...,wow,wry,wutcha,www,ya,yay,yer,yes,yrs,zac
68,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
92,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data = data_dtm.transpose()
data.head()

,68,40,29,41,56,88,49,58,24,3,...,15,76,11,25,55,53,80,70,92,94
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1st,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
50,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5am,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Find the top 20 words said by each tweeter
top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(20)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{68: [('got', 1),
  ('day', 1),
  ('bummer', 1),
  ('carr', 1),
  ('awww', 1),
  ('david', 1),
  ('shoulda', 1),
  ('dude', 0),
  ('ears', 0),
  ('film', 0),
  ('finally', 0),
  ('fine', 0),
  ('fly', 0),
  ('fml', 0),
  ('follow', 0),
  ('forever', 0),
  ('forget', 0),
  ('forgotten', 0),
  ('friend', 0),
  ('friends', 0)],
 40: [('school', 1),
  ('result', 1),
  ('today', 1),
  ('texting', 1),
  ('update', 1),
  ('facebook', 1),
  ('upset', 1),
  ('blah', 1),
  ('fml', 0),
  ('zac', 0),
  ('forever', 0),
  ('fly', 0),
  ('fine', 0),
  ('finally', 0),
  ('film', 0),
  ('file', 0),
  ('feels', 0),
  ('follow', 0),
  ('friend', 0),
  ('forget', 0)],
 29: [('dived', 1),
  ('bounds', 1),
  ('rest', 1),
  ('ball', 1),
  ('managed', 1),
  ('save', 1),
  ('times', 1),
  ('50', 1),
  ('glad', 0),
  ('forget', 0),
  ('feels', 0),
  ('file', 0),
  ('film', 0),
  ('going', 0),
  ('finally', 0),
  ('fine', 0),
  ('fly', 0),
  ('fml', 0),
  ('follow', 0),
  ('forever', 0)],
 41: [('body', 1),
  ('

In [9]:
# Print the top 15 words said by each tweeter
for x, top_words in top_dict.items():
    print(x)
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')

68
got, day, bummer, carr, awww, david, shoulda, dude, ears, film, finally, fine, fly, fml
---
40
school, result, today, texting, update, facebook, upset, blah, fml, zac, forever, fly, fine, finally
---
29
dived, bounds, rest, ball, managed, save, times, 50, glad, forget, feels, file, film, going
---
41
body, feels, like, itchy, fly, forget, forever, follow, fml, zac, fine, friend, finally, film
---
56
mad, behaving, zac, friends, feels, file, film, finally, fine, fly, fml, follow, forever, forget
---
88
crew, zac, feed, feeling, feels, file, film, finally, fine, fly, fml, follow, forever, forget
---
49
hug, need, feed, feeling, feels, file, film, finally, fine, fly, fml, follow, forever, forget
---
58
bit, thanks, yes, rains, lol, long, time, hey, fine, forever, follow, fml, fly, zac
---
24
didn, nope, zac, friend, feeling, feels, file, film, finally, fine, fly, fml, follow, forever
---
3
muera, que, zac, feel, feels, file, film, finally, fine, fly, fml, follow, forever, forget
---
37

In [10]:
# Look at the most common top words --> add them to the stop word list
# Let's first pull out the top 30 words for each comedian
words = []
for x in data.columns:
    top = [word for (word, count) in top_dict[x]]
    for t in top:
        words.append(t)
        
words

['got',
 'day',
 'bummer',
 'carr',
 'awww',
 'david',
 'shoulda',
 'dude',
 'ears',
 'film',
 'finally',
 'fine',
 'fly',
 'fml',
 'follow',
 'forever',
 'forget',
 'forgotten',
 'friend',
 'friends',
 'school',
 'result',
 'today',
 'texting',
 'update',
 'facebook',
 'upset',
 'blah',
 'fml',
 'zac',
 'forever',
 'fly',
 'fine',
 'finally',
 'film',
 'file',
 'feels',
 'follow',
 'friend',
 'forget',
 'dived',
 'bounds',
 'rest',
 'ball',
 'managed',
 'save',
 'times',
 '50',
 'glad',
 'forget',
 'feels',
 'file',
 'film',
 'going',
 'finally',
 'fine',
 'fly',
 'fml',
 'follow',
 'forever',
 'body',
 'feels',
 'like',
 'itchy',
 'fly',
 'forget',
 'forever',
 'follow',
 'fml',
 'zac',
 'fine',
 'friend',
 'finally',
 'film',
 'file',
 'feeling',
 'forgotten',
 'friends',
 'feed',
 'game',
 'mad',
 'behaving',
 'zac',
 'friends',
 'feels',
 'file',
 'film',
 'finally',
 'fine',
 'fly',
 'fml',
 'follow',
 'forever',
 'forget',
 'forgotten',
 'friend',
 'game',
 'feel',
 'gear',
 'ge

In [11]:
# Let's aggregate this list and identify the most common words along with how many routines they occur in
Counter(words).most_common()

[('film', 99),
 ('finally', 98),
 ('fine', 98),
 ('file', 95),
 ('fly', 94),
 ('fml', 91),
 ('follow', 90),
 ('forever', 83),
 ('feels', 83),
 ('forget', 75),
 ('feeling', 62),
 ('forgotten', 60),
 ('friend', 58),
 ('zac', 56),
 ('friends', 43),
 ('feel', 39),
 ('game', 33),
 ('feed', 27),
 ('gear', 24),
 ('gets', 14),
 ('favorite', 11),
 ('glad', 10),
 ('just', 9),
 ('day', 8),
 ('like', 7),
 ('girls', 7),
 ('gmt', 7),
 ('going', 6),
 ('really', 6),
 ('tomorrow', 6),
 ('sad', 6),
 ('don', 6),
 ('think', 6),
 ('got', 5),
 ('today', 5),
 ('time', 5),
 ('miss', 5),
 ('hate', 5),
 ('sorry', 5),
 ('need', 4),
 ('hey', 4),
 ('didn', 4),
 ('oh', 4),
 ('sleep', 4),
 ('won', 4),
 ('work', 4),
 ('girlfriend', 4),
 ('getting', 4),
 ('girl', 4),
 ('bad', 4),
 ('thought', 3),
 ('watch', 3),
 ('did', 3),
 ('ve', 3),
 ('doucheclown', 3),
 ('drinking', 3),
 ('class', 3),
 ('good', 3),
 ('want', 3),
 ('bed', 3),
 ('know', 3),
 ('gonna', 3),
 ('getty', 3),
 ('awww', 2),
 ('dude', 2),
 ('ears', 2),
 ('s

In [12]:
# If more than half of the tweets have it as a top word, exclude it from the list
add_stop_words = [word for word, count in Counter(words).most_common() if count > 6]
add_stop_words

['film',
 'finally',
 'fine',
 'file',
 'fly',
 'fml',
 'follow',
 'forever',
 'feels',
 'forget',
 'feeling',
 'forgotten',
 'friend',
 'zac',
 'friends',
 'feel',
 'game',
 'feed',
 'gear',
 'gets',
 'favorite',
 'glad',
 'just',
 'day',
 'like',
 'girls',
 'gmt']

In [13]:
# Add new stop words
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate document-term matrix
cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(df_main.text)
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = df_main.index

data_stop

,10,12,1st,50,5am,92,able,account,actually,added,...,worse,wow,wry,wutcha,www,ya,yay,yer,yes,yrs
68,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
92,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Try 1 

In [14]:
tdm = data_stop.transpose()
tdm.head()

,68,40,29,41,56,88,49,58,24,3,...,15,76,11,25,55,53,80,70,92,94
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1st,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
50,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5am,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(tdm.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cv.vocabulary_.items())

In [16]:
# Let's check with 5 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.038*"comfortable" + 0.038*"bounds" + 0.034*"account" + 0.031*"cool" + 0.031*"carr" + 0.031*"12" + 0.030*"counts" + 0.026*"cameron" + 0.026*"bed" + 0.024*"ahh"'),
 (1,
  '0.062*"cause" + 0.058*"came" + 0.050*"awww" + 0.043*"boo" + 0.036*"babe" + 0.032*"break" + 0.028*"champ" + 0.028*"blah" + 0.028*"ball" + 0.028*"ago"'),
 (2,
  '0.048*"blagh" + 0.039*"away" + 0.039*"burnt" + 0.035*"crashed" + 0.035*"1st" + 0.035*"bit" + 0.031*"com" + 0.027*"10" + 0.022*"class" + 0.022*"cigarette"'),
 (3,
  '0.041*"birthday" + 0.037*"constants" + 0.034*"couldn" + 0.034*"behaving" + 0.034*"baked" + 0.031*"allllll" + 0.028*"arms" + 0.027*"comments" + 0.027*"breaking" + 0.027*"blood"')]

# try 2

In [17]:
# Let's create a function to pull out nouns from a string of text

def nouns(text):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [18]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [19]:
# Apply the nouns function to the transcripts to filter only on nouns
nltk.download('averaged_perceptron_tagger')
data_nouns_adj = pd.DataFrame(df_main.text.apply(nouns_adj))
data_nouns_adj

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\FinTech\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text
68,awww bummer david carr third day
40,upset facebook result school today
29,i many times ball rest bounds
41,whole body feels fire
56,i m mad i
...,...
53,strider sick little puppy
80,rylee grace wana steve s party easter i b able
70,hey bracket pools bad one money
92,t i work


In [20]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.text)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,able,account,adidas,afternoon,aim,allllll,anaheim,andy,arms,asian,...,work,worse,wow,wry,wutcha,www,ya,yay,yer,yrs
68,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [21]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [22]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.022*"time" + 0.022*"class" + 0.016*"sorry" + 0.012*"bad" + 0.012*"nite" + 0.012*"good" + 0.012*"meeting" + 0.012*"job" + 0.012*"oh" + 0.012*"right"'),
 (1,
  '0.020*"sad" + 0.014*"tomorrow" + 0.014*"sick" + 0.014*"awww" + 0.014*"im" + 0.014*"hour" + 0.014*"party" + 0.008*"sure" + 0.008*"money" + 0.008*"wage"'),
 (2,
  '0.025*"work" + 0.014*"today" + 0.014*"broadband" + 0.014*"leslie" + 0.014*"hug" + 0.014*"body" + 0.014*"mad" + 0.014*"people" + 0.010*"gym" + 0.008*"ok"'),
 (3,
  '0.021*"sad" + 0.014*"tomorrow" + 0.014*"hey" + 0.014*"today" + 0.014*"bit" + 0.014*"sleep" + 0.014*"school" + 0.014*"night" + 0.008*"lol" + 0.008*"phone"')]

## Identify Topics in Each tweet

In [23]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=70)
ldana.print_topics()

[(0,
  '0.023*"tomorrow" + 0.023*"hey" + 0.018*"bad" + 0.012*"party" + 0.012*"nite" + 0.012*"bit" + 0.012*"lol" + 0.012*"leslie" + 0.012*"heart" + 0.012*"mad"'),
 (1,
  '0.026*"sad" + 0.019*"today" + 0.013*"sleep" + 0.013*"hug" + 0.007*"school" + 0.007*"lot" + 0.007*"money" + 0.007*"night" + 0.007*"min" + 0.007*"jobs"'),
 (2,
  '0.019*"work" + 0.013*"class" + 0.013*"gym" + 0.013*"oh" + 0.013*"phone" + 0.013*"people" + 0.007*"im" + 0.007*"today" + 0.007*"awww" + 0.007*"job"'),
 (3,
  '0.020*"time" + 0.020*"sorry" + 0.014*"sick" + 0.014*"broadband" + 0.008*"right" + 0.008*"new" + 0.008*"ok" + 0.008*"class" + 0.008*"bed" + 0.008*"hour"')]

### These four seems to be the theme.
* Topic 0: time
* Topic 1: sad
* Topic 2: work, job
* Topic 3: appology